In [1]:
import QuantLib as ql
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from sklearn.metrics import mean_absolute_error 
import QuantLib as ql
import numpy as np

In [2]:
# from scipy.stats import norm
# import numpy as np

# def find_strike_from_delta(S0, T, r, sigma, delta, right):
#     if right == "C":
#         d1 = norm.ppf(delta * np.exp(r * T))
#     else:
#         d1 = norm.ppf(1 - delta * np.exp(r * T))
#     strike = S0 * np.exp(-d1 * sigma * np.sqrt(T) + ((sigma**2) / 2) * T)
#     return strike


# S0 = 5247.5  # Spot price
# T = 30/365   # Time to expiration (in years)
# r = 0.06  # Risk-free interest rate
# sigma = 0.123788190770614  # Implied volatility
# delta = 0.474734352376905 # Delta
# right = "C"  # Call option

# strike = find_strike_from_delta(S0, T, r, sigma, delta, right)
# print("Strike price:", strike)


In [3]:


# Using forward slashes
# df1 = pd.read_excel("C:/Users/IBISP/Downloads/sp_chian.xlsx")

# Or using raw string literals
df1 = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="7")
df2 = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="14")
df3 = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="30")
df4 = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="45")
df5 = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="60")



In [4]:
arr_delta_7 =  [0.10333066863286,0.247301269636952,0.501479253877894,0.744883823219986,   0.9014347111305]
arr_volatility_7 =  [ 0.104545998592648 ,0.107588301162454 , 0.114244790636233 , 0.124719600305455 , 0.141653907687411 ]

arr_delta_14 = [ 0.102604565	, 0.253193607 , 	0.497822727	, 0.745035871, 0.909054386503881]
arr_volatility_14 = [0.105335153  , 0.109647492 , 0.117701221 ,0.133078592,	0.163695446162443]

arr_delta_30 = [0.10177538423678 , 0.259835630825842 ,0.49938322027473 , 0.747229742128656 , 0.899755717855416  ]
arr_volatility_30 = [0.108065276902309 ,  0.113062359205094 ,0.122524956552977 , 0.14068756252998,   0.172130686873854  ]

arr_delta_45 = [ 0.103275279721047 ,0.256415399	 ,0.501912955	, 0.747848225 , 0.898519269		 ]
arr_volatility_45 = [0.109516715182632 ,0.114170763 , 0.124770767 , 0.144667475 , 0.180421133  ]

arr_delta_60 = [0.0979545112394116,0.247074944763777,0.504187050700219,  0.750927577333952 , 0.900459730194448 ]
arr_volatility_60 = [0.1084320894973, 0.112665910016102 , 0.125084182094845 , 0.148316483510968,  0.184474262892502 ]

In [5]:
def interpolation(df, arr_delta, arr_volatility):

    implied_volatility1 = df['ImpliedVolatility'].to_list()
    delta1 = df['Delta'].to_list()

    
    df_delta_volatility = pd.DataFrame({'Delta': delta1[1:], 'Volatility': implied_volatility1[1:]})
    df_delta_volatility.dropna(inplace=True)

    X = arr_delta
    Y = arr_volatility

    # Interpolation method
    log_cubic_spline = ql.LogCubicNaturalSpline(X, Y)
    log_cubic_spline_value = []

    cubic_natural_spline = ql.CubicNaturalSpline(X, Y)

    cubic_natural_spline_value = []

    log_parabolic = ql.LogParabolic(X, Y)

    log_parabolic_value = []


    # Interpolated value
    for ele in df_delta_volatility['Delta'].tolist():
        interpolated_value1 = log_cubic_spline(ele, allowExtrapolation=True)
        log_cubic_spline_value.append(interpolated_value1)

        interpolated_value2 = cubic_natural_spline(ele, allowExtrapolation=True)

        cubic_natural_spline_value.append(interpolated_value2)

        interpolated_value3 = log_parabolic(ele, allowExtrapolation=True)
        log_parabolic_value.append(interpolated_value3)

    methods = {

        'CubicNaturalSpline': cubic_natural_spline,
        'LogCubicNaturalSpline': log_cubic_spline,
        'LogParabolic': log_parabolic
    }

    xx = np.linspace(min(X), max(X), 1000)

    fig = go.Figure()

    # Add original data as scatter plot
    # fig.add_trace(go.Scatter(x=X, y=Y, mode='markers', name='Original Data'))

    # Plot interpolation methods
    for name, interp_method in methods.items():
        yy = [interp_method(x, allowExtrapolation=False) for x in xx]
        fig.add_trace(go.Scatter(x=xx, y=yy, mode='lines', name=name))
    fig.add_trace(go.Scatter(x=arr_delta , y=arr_volatility, mode='markers', name='Original Data'))
    fig.add_trace(go.Scatter(x=df_delta_volatility['Delta'] , y=df_delta_volatility['Volatility'], mode='markers', name='All Original Data'))
    fig.update_layout(title='Interpolation Methods Comparison',
                    xaxis_title='X',
                    yaxis_title='Y')

    fig.show()


    df_delta_volatility['LogCubicSpline'] = log_cubic_spline_value
    df_delta_volatility['CubicNaturalSpline'] = cubic_natural_spline_value
    df_delta_volatility['LogParabolic'] = log_parabolic_value

    df_delta_volatility['combined'] = (df_delta_volatility['LogCubicSpline'] + df_delta_volatility['CubicNaturalSpline'] + df_delta_volatility['LogParabolic'])/3

    df_delta_volatility_log_cubic_spline_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['LogCubicSpline'])**2))/len(df_delta_volatility['Volatility']))
    df_delta_volatility_cubic_natural_spline_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['CubicNaturalSpline'])**2))/len(df_delta_volatility['Volatility']))
    df_delta_volatility_log_parabolic_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['LogParabolic'])**2))/len(df_delta_volatility['Volatility']))
    df_delta_volatility_combined_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['combined'])**2))/len(df_delta_volatility['Volatility']))

    df_delta_volatility_log_cubic_spline_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['LogCubicSpline'])
    df_delta_volatility_cubic_natural_spline_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['CubicNaturalSpline'])
    df_delta_volatility_log_parabolic_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['LogParabolic'])
    df_delta_volatility_combined_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['combined'])

    rmse_array = np.array([df_delta_volatility_log_cubic_spline_rmse, df_delta_volatility_cubic_natural_spline_rmse, df_delta_volatility_log_parabolic_rmse, df_delta_volatility_combined_rmse])
    mae_array = np.array([df_delta_volatility_log_cubic_spline_mae, df_delta_volatility_cubic_natural_spline_mae, df_delta_volatility_log_parabolic_mae, df_delta_volatility_combined_mae])
    
    return rmse_array, mae_array



In [6]:
rmse_array1, mae_array1 = interpolation(df1, arr_delta_7, arr_volatility_7)
rmse_array2, mae_array2 = interpolation(df2, arr_delta_14, arr_volatility_14)
rmse_array3, mae_array3 = interpolation(df3, arr_delta_30, arr_volatility_30)
rmse_array4, mae_array4 = interpolation(df4, arr_delta_45, arr_volatility_45)
rmse_array5, mae_array5 = interpolation(df5, arr_delta_60, arr_volatility_60)


In [7]:
result = pd.DataFrame({ 'Method' : ['LogCubicSpline', 'CubicNaturalSpline', 'LogParabolic', 'Combined'],
                        'RMSE7' : rmse_array1,
                        'RMSE14' : rmse_array2,
                        'RMSE30' : rmse_array3,
                        'RMSE45' : rmse_array4,
                        'MAE7' : mae_array1,
                        'MAE14' : mae_array2,
                        'MAE30' : mae_array3,
                        'MAE45' : mae_array4
                        })
result

,Method,RMSE7,RMSE14,RMSE30,RMSE45,MAE7,MAE14,MAE30,MAE45
0,LogCubicSpline,0.001067,0.003888,0.001606,0.001361,0.000729,0.001860,0.001123,0.000906
1,CubicNaturalSpline,0.001133,0.004021,0.001842,0.001727,0.000783,0.002045,0.001320,0.001165
2,LogParabolic,0.000980,0.003778,0.001397,0.001120,0.000667,0.001693,0.000948,0.000729
3,Combined,0.001057,0.003891,0.001610,0.001398,0.000723,0.001860,0.001121,0.000922


## ZW

In [8]:
dfz1 = pd.read_excel(r"C:\Users\IBISP\Downloads\ZW Option Chain(MIV).xlsx", sheet_name="APR24")
dfz2 = pd.read_excel(r"C:\Users\IBISP\Downloads\ZW Option Chain(MIV).xlsx", sheet_name="MAY24")
dfz3 = pd.read_excel(r"C:\Users\IBISP\Downloads\ZW Option Chain(MIV).xlsx", sheet_name="JUN24")
dfz4 = pd.read_excel(r"C:\Users\IBISP\Downloads\ZW Option Chain(MIV).xlsx", sheet_name="AUG24")

In [9]:
arr_delta1 =  [0.100367576259687,0.263989653869718,0.525358434176222,0.770278396538882,   0.914072934496419]
arr_volatility1 =  [0.37475235129697 ,0.343513122519804 , 0.322212611923678 , 0.311332256773131 , 0.327408203350171 ]

In [10]:
arr_delta2 = [ 0.107886593213257 , 0.251634441241605  , 0.500099698373733 , 0.766080156326088 , 0.909405887195697]
arr_volatility2 = [0.373443006706383  , 0.343854210232335 , 0.321961140156943 ,0.308048322682323 , 0.320733348022407 ]

In [11]:
arr_delta3 = [ 0.100323071469652 , 0.260791799080824 ,0.509313199573201 , 0.76423470324344 ,  0.908658070687755 ]
arr_volatility3 = [ 0.383583162245413 , 0.349534225526225 , 0.326193693388756 , 0.309396838413299 , 0.320240422606 ]

In [12]:
arr_delta4 = [0.100501750024694, 0.254427477771597 , 0.512420397795484 , 0.747378118165584, 0.914209932287387]
arr_volatility4 = [0.391429162667484 ,0.356036710182025 , 0.328102292914276 , 0.30859176626093 ,0.332342142544592 ]

In [13]:
def interpolation(df, arr_delta, arr_volatility):

    implied_volatility1 = df['ImpliedVolatility'].to_list()
    delta1 = df['Delta'].to_list()

    
    df_delta_volatility = pd.DataFrame({'Delta': delta1[1:], 'Volatility': implied_volatility1[1:]})
    df_delta_volatility.dropna(inplace=True)

    X = arr_delta
    Y = arr_volatility

    # Interpolation method
    log_cubic_spline = ql.LogCubicNaturalSpline(X, Y)
    log_cubic_spline_value = []

    cubic_natural_spline = ql.CubicNaturalSpline(X, Y)

    cubic_natural_spline_value = []

    log_parabolic = ql.LogParabolic(X, Y)

    log_parabolic_value = []


    # Interpolated value
    for ele in df_delta_volatility['Delta'].tolist():
        interpolated_value1 = log_cubic_spline(ele, allowExtrapolation=True)
        log_cubic_spline_value.append(interpolated_value1)

        interpolated_value2 = cubic_natural_spline(ele, allowExtrapolation=True)

        cubic_natural_spline_value.append(interpolated_value2)

        interpolated_value3 = log_parabolic(ele, allowExtrapolation=True)
        log_parabolic_value.append(interpolated_value3)

    methods = {

        'CubicNaturalSpline': cubic_natural_spline,
        'LogCubicNaturalSpline': log_cubic_spline,
        'LogParabolic': log_parabolic
    }

    xx = np.linspace(min(X), max(X), 1000)

    fig = go.Figure()

    # Add original data as scatter plot
    # fig.add_trace(go.Scatter(x=X, y=Y, mode='markers', name='Original Data'))

    # Plot interpolation methods
    for name, interp_method in methods.items():
        yy = [interp_method(x, allowExtrapolation=False) for x in xx]
        fig.add_trace(go.Scatter(x=xx, y=yy, mode='lines', name=name))
    fig.add_trace(go.Scatter(x=arr_delta , y=arr_volatility, mode='markers', name='Original Data'))
    fig.add_trace(go.Scatter(x=df_delta_volatility['Delta'] , y=df_delta_volatility['Volatility'], mode='markers', name='All Original Data'))
    fig.update_layout(title='Interpolation Methods Comparison',
                    xaxis_title='X',
                    yaxis_title='Y')

    fig.show()


    df_delta_volatility['LogCubicSpline'] = log_cubic_spline_value
    df_delta_volatility['CubicNaturalSpline'] = cubic_natural_spline_value
    df_delta_volatility['LogParabolic'] = log_parabolic_value

    df_delta_volatility['combined'] = (df_delta_volatility['LogCubicSpline'] + df_delta_volatility['CubicNaturalSpline'] + df_delta_volatility['LogParabolic'])/3

    df_delta_volatility_log_cubic_spline_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['LogCubicSpline'])**2))/len(df_delta_volatility['Volatility']))
    df_delta_volatility_cubic_natural_spline_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['CubicNaturalSpline'])**2))/len(df_delta_volatility['Volatility']))
    df_delta_volatility_log_parabolic_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['LogParabolic'])**2))/len(df_delta_volatility['Volatility']))
    df_delta_volatility_combined_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['combined'])**2))/len(df_delta_volatility['Volatility']))

    df_delta_volatility_log_cubic_spline_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['LogCubicSpline'])
    df_delta_volatility_cubic_natural_spline_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['CubicNaturalSpline'])
    df_delta_volatility_log_parabolic_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['LogParabolic'])
    df_delta_volatility_combined_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['combined'])

    rmse_array = np.array([df_delta_volatility_log_cubic_spline_rmse, df_delta_volatility_cubic_natural_spline_rmse, df_delta_volatility_log_parabolic_rmse, df_delta_volatility_combined_rmse])
    mae_array = np.array([df_delta_volatility_log_cubic_spline_mae, df_delta_volatility_cubic_natural_spline_mae, df_delta_volatility_log_parabolic_mae, df_delta_volatility_combined_mae])
    
    return rmse_array, mae_array



In [14]:
rmse_array1, mae_array1 = interpolation(dfz1, arr_delta1, arr_volatility1)
rmse_array2, mae_array2 = interpolation(dfz2, arr_delta2, arr_volatility2)
rmse_array3, mae_array3 = interpolation(dfz3, arr_delta3, arr_volatility3)
rmse_array4, mae_array4 = interpolation(dfz4, arr_delta4, arr_volatility4)


In [22]:
result_zw = pd.DataFrame({ 'Method' : ['LogCubicSpline', 'CubicNaturalSpline', 'LogParabolic', 'Combined'],
                        'RMSEAPR' : rmse_array1,
                        'RMSEMAY' : rmse_array2,
                        'RMSEJUN' : rmse_array3,
                        'RMSEAUG' : rmse_array4,
                        'MAEAPR' : mae_array1,
                        'MAEMAY' : mae_array2,
                        'MAEJUN' : mae_array3,
                        'MAEAUG' : mae_array4})


In [24]:
result_zw

,Method,RMSEAPR,RMSEMAY,RMSEJUN,RMSEAUG,MAEAPR,MAEMAY,MAEJUN,MAEAUG
0,LogCubicSpline,0.001384,0.001926,0.001457,0.001399,0.000901,0.001306,0.000975,0.000901
1,CubicNaturalSpline,0.001430,0.002071,0.001748,0.001726,0.000966,0.001460,0.001206,0.001137
2,LogParabolic,0.001286,0.001841,0.001229,0.001176,0.000795,0.001179,0.000817,0.000770
3,Combined,0.001364,0.001939,0.001473,0.001428,0.000886,0.001305,0.000982,0.000915


## put

In [17]:
arr_delta_7 = [-0.906431805141028 , -0.751636822073632 ,-0.497458837940968	, -0.254053914008985 ,-0.103751162587272 ]
arr_volatility_7 = [ 0.106775485012301 ,0.108924353816607 , 0.115101494988724 , 0.1255158740103 ,0.142468325693439 ]

arr_delta_14 = [-0.910690389195292 ,-0.759245108937226 , -0.500076338766803 ,-0.252863194945459 ,-0.0957598102103778 ]
arr_volatility_14 = [0.103391037150505 ,0.108849808285943 ,0.117710169077046 ,  0.132516647070948 ,0.159227452854588 ]

arr_delta_30 =  [-0.0981864757127743 ,-0.248313957825809,-0.49616047974473 , -0.755441580233471 , -0.904512075042767 ][::-1]
arr_volatility_30 = [ 0.172264353450903 ,0.141238111647596 ,0.122752373058901 , 0.112447044596787  ,0.108226994156244 ][::-1]

arr_delta_45 = [-0.898782661737442 ,-0.752842744069179 , -0.491447274636865 , -0.250671240317813 , -0.0985491869139686 ]
arr_volatility_45 = [0.108495780316165 ,0.113322163536464 , 0.12444698731072 , 0.144289328269737 ,0.179105668565425 ]

arr_delta_60 = [ -0.901576548182131 ,-0.756904232082295 , -0.500601622160788 , -0.249096879058567 ,-0.10089081105517 ] 
arr_volatility_60 = [0.108323800620372 , 0.112663803238954 , 0.124313520180007, 0.147511863154295, 0.181909472223689 ]

In [18]:
dfp1 = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="7")
dfp2 = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="14")
dfp3 = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="30")
dfp4 = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="45")
dfp5 = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="60")

In [19]:
def interpolation(df, arr_delta, arr_volatility):

    implied_volatility1 = df['ImpliedVolatility.1'].to_list()
    delta1 = df['Delta.1'].to_list()

    
    df_delta_volatility = pd.DataFrame({'Delta': delta1[1:], 'Volatility': implied_volatility1[1:]})
    df_delta_volatility.dropna(inplace=True)

    X = arr_delta
    Y = arr_volatility

    # Interpolation method
    log_cubic_spline = ql.LogCubicNaturalSpline(X, Y)
    log_cubic_spline_value = []

    cubic_natural_spline = ql.CubicNaturalSpline(X, Y)

    cubic_natural_spline_value = []

    log_parabolic = ql.LogParabolic(X, Y)

    log_parabolic_value = []


    # Interpolated value
    for ele in df_delta_volatility['Delta'].tolist():
        interpolated_value1 = log_cubic_spline(ele, allowExtrapolation=True)
        log_cubic_spline_value.append(interpolated_value1)

        interpolated_value2 = cubic_natural_spline(ele, allowExtrapolation=True)

        cubic_natural_spline_value.append(interpolated_value2)

        interpolated_value3 = log_parabolic(ele, allowExtrapolation=True)
        log_parabolic_value.append(interpolated_value3)

    methods = {

        'CubicNaturalSpline': cubic_natural_spline,
        'LogCubicNaturalSpline': log_cubic_spline,
        'LogParabolic': log_parabolic
    }

    xx = np.linspace(min(X), max(X), 1000)

    fig = go.Figure()

    # Add original data as scatter plot
    # fig.add_trace(go.Scatter(x=X, y=Y, mode='markers', name='Original Data'))

    # Plot interpolation methods
    for name, interp_method in methods.items():
        yy = [interp_method(x, allowExtrapolation=True) for x in xx]
        fig.add_trace(go.Scatter(x=xx, y=yy, mode='lines', name=name))
    fig.add_trace(go.Scatter(x=arr_delta , y=arr_volatility, mode='markers', name='Original Data'))
    fig.add_trace(go.Scatter(x=df_delta_volatility['Delta'] , y=df_delta_volatility['Volatility'], mode='markers', name='All Original Data'))
    fig.update_layout(title='Interpolation Methods Comparison',
                    xaxis_title='X',
                    yaxis_title='Y')

    fig.show()


    df_delta_volatility['LogCubicSpline'] = log_cubic_spline_value
    df_delta_volatility['CubicNaturalSpline'] = cubic_natural_spline_value
    df_delta_volatility['LogParabolic'] = log_parabolic_value

    df_delta_volatility['combined'] = (df_delta_volatility['LogCubicSpline'] + df_delta_volatility['CubicNaturalSpline'] + df_delta_volatility['LogParabolic'])/3

    df_delta_volatility_log_cubic_spline_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['LogCubicSpline'])**2))/len(df_delta_volatility['Volatility']))
    df_delta_volatility_cubic_natural_spline_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['CubicNaturalSpline'])**2))/len(df_delta_volatility['Volatility']))
    df_delta_volatility_log_parabolic_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['LogParabolic'])**2))/len(df_delta_volatility['Volatility']))
    df_delta_volatility_combined_rmse = np.sqrt((sum((df_delta_volatility['Volatility'] - df_delta_volatility['combined'])**2))/len(df_delta_volatility['Volatility']))

    df_delta_volatility_log_cubic_spline_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['LogCubicSpline'])
    df_delta_volatility_cubic_natural_spline_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['CubicNaturalSpline'])
    df_delta_volatility_log_parabolic_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['LogParabolic'])
    df_delta_volatility_combined_mae = mean_absolute_error(df_delta_volatility['Volatility'], df_delta_volatility['combined'])

    rmse_array = np.array([df_delta_volatility_log_cubic_spline_rmse, df_delta_volatility_cubic_natural_spline_rmse, df_delta_volatility_log_parabolic_rmse, df_delta_volatility_combined_rmse])
    mae_array = np.array([df_delta_volatility_log_cubic_spline_mae, df_delta_volatility_cubic_natural_spline_mae, df_delta_volatility_log_parabolic_mae, df_delta_volatility_combined_mae])
    
    return rmse_array, mae_array



In [20]:
rmse_array1, mae_array1 = interpolation(dfp1, arr_delta_7, arr_volatility_7)
rmse_array2, mae_array2 = interpolation(dfp2, arr_delta_14, arr_volatility_14)
rmse_array3, mae_array3 = interpolation(dfp3, arr_delta_30, arr_volatility_30)
rmse_array4, mae_array4 = interpolation(dfp4, arr_delta_45, arr_volatility_45)
rmse_array5, mae_array5 = interpolation(dfp5, arr_delta_60, arr_volatility_60)

In [21]:
result_put = pd.DataFrame({ 'Method' : ['LogCubicSpline', 'CubicNaturalSpline', 'LogParabolic', 'Combined'],
                        'RMSE7' : rmse_array1,
                        'RMSE14' : rmse_array2,
                        'RMSE30' : rmse_array3,
                        'RMSE45' : rmse_array4,
                        'MAE7' : mae_array1,
                        'MAE14' : mae_array2,
                        'MAE30' : mae_array3,
                        'MAE45' : mae_array4
                        })
result_put

,Method,RMSE7,RMSE14,RMSE30,RMSE45,MAE7,MAE14,MAE30,MAE45
0,LogCubicSpline,0.001384,0.001926,0.001457,0.001399,0.000901,0.001306,0.000975,0.000901
1,CubicNaturalSpline,0.001430,0.002071,0.001748,0.001726,0.000966,0.001460,0.001206,0.001137
2,LogParabolic,0.001286,0.001841,0.001229,0.001176,0.000795,0.001179,0.000817,0.000770
3,Combined,0.001364,0.001939,0.001473,0.001428,0.000886,0.001305,0.000982,0.000915
